In [2]:
import pandas as pd
import numpy as np

# change path to your local file
df = pd.read_csv("Superstore.csv", encoding="latin1")

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [1]:
import pandas as pd
import numpy as np

# Load fresh (important)
df = pd.read_csv("Superstore.csv", encoding="latin1")

# Standardize column names
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace("-", "_")

# Parse dates (your file is DD-MM-YYYY)
df["Order_Date"] = pd.to_datetime(df["Order_Date"], format="%d-%m-%Y", errors="coerce")
df["Ship_Date"]  = pd.to_datetime(df["Ship_Date"],  format="%d-%m-%Y", errors="coerce")

# Verify
print(df[["Order_Date","Ship_Date"]].dtypes)
print("NaT counts:", df["Order_Date"].isna().sum(), df["Ship_Date"].isna().sum())
df[["Order_Date","Ship_Date"]].head()


Order_Date    datetime64[ns]
Ship_Date     datetime64[ns]
dtype: object
NaT counts: 0 0


,Order_Date,Ship_Date
0,2013-11-09,2013-11-12
1,2013-11-09,2013-11-12
2,2013-06-13,2013-06-17
3,2012-10-11,2012-10-18
4,2012-10-11,2012-10-18


In [2]:
df["Order_Year"]  = df["Order_Date"].dt.year
df["Order_Month"] = df["Order_Date"].dt.month
df["Order_Quarter"] = df["Order_Date"].dt.to_period("Q").astype(str)
df["YearMonth"]   = df["Order_Date"].dt.to_period("M").astype(str)

# Shipping days (useful operational KPI)
df["Ship_Days"] = (df["Ship_Date"] - df["Order_Date"]).dt.days
df[["Order_Date","Ship_Date","Ship_Days","YearMonth"]].head()


,Order_Date,Ship_Date,Ship_Days,YearMonth
0,2013-11-09,2013-11-12,3,2013-11
1,2013-11-09,2013-11-12,3,2013-11
2,2013-06-13,2013-06-17,4,2013-06
3,2012-10-11,2012-10-18,7,2012-10
4,2012-10-11,2012-10-18,7,2012-10


## Data validation checks

In [3]:
print("Ship before Order:", (df["Ship_Days"] < 0).sum())
print("Very long shipping (>30 days):", (df["Ship_Days"] > 30).sum())
df["Ship_Days"].describe()


Ship before Order: 0
Very long shipping (>30 days): 0


count    9994.000000
mean        3.958875
std         1.747097
min         0.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: Ship_Days, dtype: float64

## Create “analysis tables”
1. Monthly summary

In [4]:
monthly = (df.groupby("YearMonth", as_index=False)
             .agg(Total_Revenue=("Sales","sum"),
                  Total_Profit=("Profit","sum"),
                  Orders=("Order_ID","nunique")))
monthly["Profit_Margin"] = monthly["Total_Profit"] / monthly["Total_Revenue"]
monthly.head()


,YearMonth,Total_Revenue,Total_Profit,Orders,Profit_Margin
0,2011-01,13946.229,2446.7711,31,0.175443
1,2011-02,4810.558,865.7280,29,0.179964
2,2011-03,55691.009,498.7299,71,0.008955
3,2011-04,28295.345,3488.8352,66,0.123301
4,2011-05,23648.287,2738.7096,69,0.115810


2. Region summary

In [5]:
region = (df.groupby("Region", as_index=False)
            .agg(Total_Revenue=("Sales","sum"),
                 Total_Profit=("Profit","sum"),
                 Orders=("Order_ID","nunique")))
region["Profit_Margin"] = region["Total_Profit"] / region["Total_Revenue"]
region.sort_values("Total_Profit", ascending=False).head()


,Region,Total_Revenue,Total_Profit,Orders,Profit_Margin
3,West,725457.8245,108418.4489,1611,0.149448
1,East,678781.2400,91522.7800,1401,0.134834
2,South,391721.9050,46749.4303,822,0.119343
0,Central,501239.8908,39706.3625,1175,0.079216


3. Top/Bottom products

In [6]:
product_profit = (df.groupby("Product_Name", as_index=False)
                    .agg(Total_Profit=("Profit","sum"),
                         Total_Revenue=("Sales","sum")))
top10 = product_profit.sort_values("Total_Profit", ascending=False).head(10)
bottom10 = product_profit.sort_values("Total_Profit", ascending=True).head(10)

top10, bottom10


(                                           Product_Name  Total_Profit  \
 403               Canon imageCLASS 2200 Advanced Copier    25199.9280   
 648   Fellowes PB500 Electric Punch Plastic Comb Bin...     7753.0390   
 803                Hewlett Packard LaserJet 3310 Copier     6983.8836   
 399                  Canon PC1060 Personal Laser Copier     4570.9347   
 785   HP Designjet T520 Inkjet Large Format Printer ...     4094.9766   
 164                   Ativa V4110MDD Micro-Cut Shredder     3772.9461   
 18     3D Systems Cube Printer, 2nd Generation, Magenta     3717.9714   
 1274  Plantronics Savi W720 Multi-Device Wireless He...     3696.2820   
 893                Ibico EPK-21 Electric Binding System     3345.2823   
 1831                  Zebra ZM400 Thermal Label Printer     3343.5360   
 
       Total_Revenue  
 403       61599.824  
 648       27453.384  
 803       18839.686  
 399       11619.834  
 785       18374.895  
 164        7699.890  
 18        14299.890  


## Export cleaned dataset + summary tables

In [7]:
df.to_csv("superstore_cleaned.csv", index=False)
monthly.to_csv("monthly_summary.csv", index=False)
region.to_csv("region_summary.csv", index=False)
top10.to_csv("top10_products_profit.csv", index=False)
bottom10.to_csv("bottom10_products_profit.csv", index=False)

print("Exports created!")


Exports created!
